In [7]:
import os
os.chdir("../")

In [8]:
import os
os.getcwd()

'd:\\projects\\DeepLearningCnnClassifier'

In [9]:
from dataclasses import dataclass
from pathlib import Path

In [27]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path :Path
    updated_base_model :Path
    params_image_size:list
    params_learning_rate:float
    params_include_top:bool
    params_weights:str
    params_classes:int


In [28]:
from DeepClassifier.constants import *
from DeepClassifier.utils import read_yaml, create_directories


In [37]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        print(self.config)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
        root_dir = Path(config.root_dir),
        base_model_path = Path(config.base_model_path),
        params_image_size = self.params.IMAGE_SIZE,
        params_learning_rate=self.params.LEARNING_RATE,
        params_include_top=self.params.INCLUDE_TOP,
        params_weights=self.params.WEIGHTS,
        params_classes=self.params.CLASSES
        )
        return prepare_base_model_config


In [30]:
#config = read_yaml(config_filepath)

In [31]:
import os
from zipfile import ZipFile
import tensorflow as tf

In [32]:
import urllib.request as request

In [33]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [38]:
config = ConfigurationManager()

{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/prepare_base_model', 'source_URL': 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion', 'prepare_base_model': None, 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}}


In [39]:
config.get_prepare_base_model_config

<bound method ConfigurationManager.get_prepare_base_model_config of <__main__.ConfigurationManager object at 0x0000027E2C245F10>>

In [36]:
#prepare_base_model = PrepareBaseModel(config=ConfigurationManager().get_prepare_base_model_config())

In [40]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/prepare_base_model', 'source_URL': 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion', 'prepare_base_model': None, 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}}


AttributeError: 'function' object has no attribute 'params_image_size'